## Definição da API que irá receber as requisições

In [ ]:
from flask import Flask, jsonify, request, Response
from flask import render_template
from pymongo import MongoClient
import ast
from pprintpp import pprint
import json
from bson import json_util

app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = True
app.config['JSON_SORT_KEYS'] = False

labels = []
values = []

# Define uma classe para utilização da conexão com o MongoDB
class MongoDB:
    
    def __init__(self, dbname):
        self._conn = MongoClient("localhost", 27017)
        self._db   = self._conn['twitter']

    def getCollection(self, name="hashtags"):
        return self._db[name]
    
@app.route('/v1/twits', methods=['GET'])
def get_all():
    
    try:
        mongo = MongoDB("twitter")
        coll_hashtags = mongo.getCollection().find({})
        
        if coll_hashtags.count() == 0:
            return get_response(404, "Hashtags não encontradas!", "{}")
        
        return get_response(200, "Sucesso!", coll_hashtags)
    except Exception as ex:
        print("Error: %s" % ex)
        return get_response(500, ex, "{}")
    
@app.route('/v1/twits/<string:hashtag>', methods=['GET'])
def get_one(hashtag):
    
    try:
        mongo = MongoDB("twitter")
        coll_hashtags = mongo.getCollection().find({"hashtag": hashtag}).limit(1)
        
        if coll_hashtags.count() == 0:
            return get_response(404, "Hashtag não encontrada!", "{}")
        
        return get_response(200, "Sucesso!", coll_hashtags)
    except Exception as ex:
        print("Error: %s" % ex)
        return get_response(500, ex, "{}")


@app.route('/v1/twits', methods=['POST'])
def post(_upsert = True):
    
    if not request.json or not 'hashtag' in request.json:
        return get_response(400, "Bad Request!", "{}")
    
    try:
        mongo = MongoDB("twitter")
        upsertResult = mongo.getCollection().update_one(
                        {'hashtag': request.json['hashtag'] }, 
                        { "$inc": { "hashtag_count": request.json['hashtag_count'] } },
                          upsert = _upsert
                       )
        print("UpdatedId: " + upsertResult.upserted_id)
        if upsertResult.upserted_id == 0:
            return get_response(200, "Hashtag atualizada com sucesso!", upsertResult.raw_result)
        else:
            return get_response(201, "Hashtag inserida com sucesso!", upsertResult.raw_result)
        
    except Exception as ex:
        print("Error: %s" % ex)
        return get_response(500, ex, "{}")
    
def get_response(_http, _msg, _data):
    return jsonify(http = _http, msg = _msg, data = json.loads(json_util.dumps(_data).encode("utf-8")))


if __name__ == "__main__":
    print("[Após subir a API, iniciar o socket e contexto spark em: 02_socket_twitter]")
    app.run(host = 'localhost', port = 5001)

[Após subir a API, iniciar o socket e contexto spark em: 02_socket_twitter]
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001/ (Press CTRL+C to quit)
